Deep Learning in het Nederlands
=============

Tensorflow in Dutch
------------
In de reeks `Deep Learning in het Nederlands - Tensorflow in Dutch` maak je kennis met de methodes en technieken voor Deep Learning met Tensorflow. Experimenteer, duik in het diepe of leer bij met de 6 opdrachten. De volledige repository staat op mijn [Github](https://github.com/victorr0/Deep-Learning-Nederlands). 

TensorFlow is een open source software bibliotheek voor Machine Learning waarmee je neurale netwerken kunt bouwen en trainen om patronen en correlaties te detecteren en te ontcijferen: [Tensorflow](https://github.com/tensorflow/tensorflow.git). 

Deze opdrachten volgen het leertraject van [Udacity](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/examples/udacity) en zijn in het Engels beschikbaar in de standaard TensorFlow repository. Ik heb ze naar het Nederlands vertaald en in een repository geplaatst. 


Opdracht 1
------------

Het doel van deze opdracht is het leren van eenvoudige data verzamel praktijken en vertrouwd raken met een aantal van de gegevens die we later zullen hergebruiken. Dit notebook maakt gebruik van de [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) dataset die gebruikt wordt met python experimenten. 

Deze dataset lijkt op de klassieke [MNIST](http://yann.lecun.com/exdb/mnist/) dataset, alleen bevat meer realistischere data zoals je die in het echt tegenkomt: de taak is zwaarder en de data is veel minder 'schoon' dan MNIST.

In [0]:
# Dit zijn alle modules die nodig zijn voor dit notebook. 
# Zorg dat ze allemaal geimporteerd kunnen worden
# voordat je verder gaat.
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tarfile
from IPython.display import display, Image
from scipy import ndimage
from sklearn.linear_model import LogisticRegression
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle

# Configureer matplotlib backend om later in de opdracht inline in IPython te plotten.
%matplotlib inline

Als eerste gaan we de dataset downloaden naar onze lokale machine. De gegevens bestaan uit characters (letters A t/m J) die in verschillende lettertypes worden weergegeven in afbeeldingen van 28 x 28 pixels.

De labels zijn beperkt tot 'A' tot 'J' (10 klassen/classes). De trainingsset heeft ongeveer 500k en de testset 19000 gelabelde voorbeelden. Gezien de grootte van de set hoor je de modellen op elke computer redelijk snel te kunnen trainen.

In [0]:
url = 'https://commondatastorage.googleapis.com/books1000/'
last_percent_reported = None
data_root = '.' # Verander deze variabele om de gegevens ergens anders op te slaan.

def download_progress_hook(count, blockSize, totalSize):
  """A hook to report the progress of a download. This is mostly intended for users with
  slow internet connections. Reports every 5% change in download progress.
  """
  global last_percent_reported
  percent = int(count * blockSize * 100 / totalSize)

  if last_percent_reported != percent:
    if percent % 5 == 0:
      sys.stdout.write("%s%%" % percent)
      sys.stdout.flush()
    else:
      sys.stdout.write(".")
      sys.stdout.flush()
      
    last_percent_reported = percent
        
def maybe_download(filename, expected_bytes, force=False):
  """Download a file if not present, and make sure it's the right size."""
  dest_filename = os.path.join(data_root, filename)
  if force or not os.path.exists(dest_filename):
    print('Poging om te downloaden:', filename) 
    filename, _ = urlretrieve(url + filename, dest_filename, reporthook=download_progress_hook)
    print('\nDownload voltooid!')
  statinfo = os.stat(dest_filename)
  if statinfo.st_size == expected_bytes:
    print('Gevonden en geverifieerd', dest_filename)
  else:
    raise Exception(
      'Kon het volgende niet verifiëren: ' + dest_filename + '. Kun je het handmatig met een browser downloaden?')
  return dest_filename

train_filename = maybe_download('notMNIST_large.tar.gz', 247336696)
test_filename = maybe_download('notMNIST_small.tar.gz', 8458043)

Found and verified notMNIST_large.tar.gz
Found and verified notMNIST_small.tar.gz


Pak de dataset in het .tar.gz bestand uit.
Je hebt nu een aantal directories gelabeled van A tot en met J.

In [0]:
num_classes = 10
np.random.seed(133)

def maybe_extract(filename, force=False):
  root = os.path.splitext(os.path.splitext(filename)[0])[0]  # Verwijder .tar.gz
  if os.path.isdir(root) and not force:
    # Je kunt dit overriden door het veranderen van force=False naar force = True.
    print('%s Al aanwezig - Uitpakken overgeslagen voor %s.' % (root, filename))
  else:
    print('Data uitpakken voor %s. Dit kan een tijdje duren. Even geduld aub.' % root)
    tar = tarfile.open(filename)
    sys.stdout.flush()
    tar.extractall(data_root)
    tar.close()
  data_folders = [
    os.path.join(root, d) for d in sorted(os.listdir(root))
    if os.path.isdir(os.path.join(root, d))]
  if len(data_folders) != num_classes:
    raise Exception(
      'Verwachtte %d mappen, 1 per class. Echter zijn er %d gevonden.' % (
        num_classes, len(data_folders)))
  print(data_folders)
  return data_folders
  
train_folders = maybe_extract(train_filename)
test_folders = maybe_extract(test_filename)

['notMNIST_large/A', 'notMNIST_large/B', 'notMNIST_large/C', 'notMNIST_large/D', 'notMNIST_large/E', 'notMNIST_large/F', 'notMNIST_large/G', 'notMNIST_large/H', 'notMNIST_large/I', 'notMNIST_large/J']
['notMNIST_small/A', 'notMNIST_small/B', 'notMNIST_small/C', 'notMNIST_small/D', 'notMNIST_small/E', 'notMNIST_small/F', 'notMNIST_small/G', 'notMNIST_small/H', 'notMNIST_small/I', 'notMNIST_small/J']


---
Probleem/vraag 1
---------

Laten we eens kijken naar de data om te checken hoe het er uitziet. Elk exemplar (voorbeeld) hoort een afbeelding te zijn van een letter A tot en met J in een ander lettertype. Laat een voorbeeld zien van de afbeeldingen die we zojuist hebben gedownload. 

Hint: Gebruik hiervoor de module IPython.display.

---

Nu laden we de data in een praktischer formaat. Omdat afhankelijk van de computerinstellingen misschien niet alle data in het geheugen past, laden we elke class in een aparte dataset en slaan die onafhankelijk van elkaar op. Later gaan we ze weer samenvoegen tot 1 overzichtelijke dataset.

We converteren de hele dataset naar een 3D-array (afbeelding-index, x, y) van floating point-waardes, genormaliseerd met een gemiddelde van ongeveer 0 en een standaardafwijking ~ 0,5. Dit maakt het training gemakkelijker. 

Sommige afbeeldingen zijn misschien niet leesbaar, in dat geval slaan we die over. 

In [0]:
image_size = 28  # Pixel breedte en hoogte.
pixel_depth = 255.0  # Aantal levels per pixel.

def load_letter(folder, min_num_images):
  """Load the data for a single letter label."""
  image_files = os.listdir(folder)
  dataset = np.ndarray(shape=(len(image_files), image_size, image_size),
                         dtype=np.float32)
  print(folder)
  num_images = 0
  for image in image_files:
    image_file = os.path.join(folder, image)
    try:
      image_data = (ndimage.imread(image_file).astype(float) - 
                    pixel_depth / 2) / pixel_depth
      if image_data.shape != (image_size, image_size):
        raise Exception('Onverwachtte afbeelding-grootte: %s' % str(image_data.shape))
      dataset[num_images, :, :] = image_data
      num_images = num_images + 1
    except IOError as e:
      print('Kon het volgende niet lezen:', image_file, ':', e, '- geen probleem, slaan we over.')
    
  dataset = dataset[0:num_images, :, :]
  if num_images < min_num_images:
    raise Exception('Veel minder afbeelding dan verwacht: %d < %d' %
                    (num_images, min_num_images))
    
  print('Volledige dataset tensor:', dataset.shape)
  print('Mean (gemiddelde):', np.mean(dataset))
  print('Standard deviation (standaard deviatie):', np.std(dataset))
  return dataset
        
def maybe_pickle(data_folders, min_num_images_per_class, force=False):
  dataset_names = []
  for folder in data_folders:
    set_filename = folder + '.pickle'
    dataset_names.append(set_filename)
    if os.path.exists(set_filename) and not force:
      # Je kunt dit overriden door het veranderen van force=False naar force = True.
      print('%s bestaat al - pickling wordt overgeslagen.' % set_filename)
    else:
      print('Pickling %s.' % set_filename)
      dataset = load_letter(folder, min_num_images_per_class)
      try:
        with open(set_filename, 'wb') as f:
          pickle.dump(dataset, f, pickle.HIGHEST_PROTOCOL)
      except Exception as e:
        print('Kon het volgende niet opslaan', set_filename, ':', e)
  
  return dataset_names

train_datasets = maybe_pickle(train_folders, 45000)
test_datasets = maybe_pickle(test_folders, 1800)

notMNIST_large/A
Could not read: notMNIST_large/A/Um9tYW5hIEJvbGQucGZi.png : cannot identify image file - it's ok, skipping.
Could not read: notMNIST_large/A/RnJlaWdodERpc3BCb29rSXRhbGljLnR0Zg==.png : cannot identify image file - it's ok, skipping.
Could not read: notMNIST_large/A/SG90IE11c3RhcmQgQlROIFBvc3Rlci50dGY=.png : cannot identify image file - it's ok, skipping.
Full dataset tensor: (52909, 28, 28)
Mean: -0.12848
Standard deviation: 0.425576
notMNIST_large/B
Could not read: notMNIST_large/B/TmlraXNFRi1TZW1pQm9sZEl0YWxpYy5vdGY=.png : cannot identify image file - it's ok, skipping.
Full dataset tensor: (52911, 28, 28)
Mean: -0.00755947
Standard deviation: 0.417272
notMNIST_large/C
Full dataset tensor: (52912, 28, 28)
Mean: -0.142321
Standard deviation: 0.421305
notMNIST_large/D
Could not read: notMNIST_large/D/VHJhbnNpdCBCb2xkLnR0Zg==.png : cannot identify image file - it's ok, skipping.
Full dataset tensor: (52911, 28, 28)
Mean: -0.0574553
Standard deviation: 0.434072
notMNIST_l

---
Probleem/vraag 2
---------

Laten we checken of de data er nog steeds goed uitzien. Toon een voorbeeld van de afbeeldingen en labels uit de ndarray. 

Hint: Gebruik hiervoor de module matplotlib.pyplot.


---

---
Probleem/vraag 3
---------
Nog een check: De gegevens horen in evenwicht te zijn tussen de verschillende klassen. Verifieer dat.

---

Knip, plak of pas de trainingsgegevens aan indien nodig. Afhankelijk van uw computerinstellingen kan de data niet allemaal tegelijkertijd in het geheugen passen. Je kunt 'train_size' aanpassen indien nodig. 

De labels (A t/m J) worden opgeslagen in een aparte array van integer getallen 0 tot en met 9.

Maak ook een validatie dataset voor hyperparameter tuning.

In [0]:
def make_arrays(nb_rows, img_size):
  if nb_rows:
    dataset = np.ndarray((nb_rows, img_size, img_size), dtype=np.float32)
    labels = np.ndarray(nb_rows, dtype=np.int32)
  else:
    dataset, labels = None, None
  return dataset, labels

def merge_datasets(pickle_files, train_size, valid_size=0):
  num_classes = len(pickle_files)
  valid_dataset, valid_labels = make_arrays(valid_size, image_size)
  train_dataset, train_labels = make_arrays(train_size, image_size)
  vsize_per_class = valid_size // num_classes
  tsize_per_class = train_size // num_classes
    
  start_v, start_t = 0, 0
  end_v, end_t = vsize_per_class, tsize_per_class
  end_l = vsize_per_class+tsize_per_class
  for label, pickle_file in enumerate(pickle_files):       
    try:
      with open(pickle_file, 'rb') as f:
        letter_set = pickle.load(f)
        # Nu schudden we de letters zodat de validatie en trainingsset helemaal willekeurig/random zijn.
        np.random.shuffle(letter_set)
        if valid_dataset is not None:
          valid_letter = letter_set[:vsize_per_class, :, :]
          valid_dataset[start_v:end_v, :, :] = valid_letter
          valid_labels[start_v:end_v] = label
          start_v += vsize_per_class
          end_v += vsize_per_class
                    
        train_letter = letter_set[vsize_per_class:end_l, :, :]
        train_dataset[start_t:end_t, :, :] = train_letter
        train_labels[start_t:end_t] = label
        start_t += tsize_per_class
        end_t += tsize_per_class
    except Exception as e:
      print('Kon het volgende niet verwerken', pickle_file, ':', e)
      raise
    
  return valid_dataset, valid_labels, train_dataset, train_labels
            
            
train_size = 200000
valid_size = 10000
test_size = 10000

valid_dataset, valid_labels, train_dataset, train_labels = merge_datasets(
  train_datasets, train_size, valid_size)
_, _, test_dataset, test_labels = merge_datasets(test_datasets, test_size)

print('Training:', train_dataset.shape, train_labels.shape)
print('Validation:', valid_dataset.shape, valid_labels.shape)
print('Testing:', test_dataset.shape, test_labels.shape)

Training (200000, 28, 28) (200000,)
Validation (10000, 28, 28) (10000,)
Testing (10000, 28, 28) (10000,)


Vervolgens maken we de dataset's volgorde willekeurig. Het is belangrijk om de labels goed te schudden voor de training en de testverdelingen.

In [0]:
def randomize(dataset, labels):
  permutation = np.random.permutation(labels.shape[0])
  shuffled_dataset = dataset[permutation,:,:]
  shuffled_labels = labels[permutation]
  return shuffled_dataset, shuffled_labels
train_dataset, train_labels = randomize(train_dataset, train_labels)
test_dataset, test_labels = randomize(test_dataset, test_labels)
valid_dataset, valid_labels = randomize(valid_dataset, valid_labels)

---
Probleem/vraag 4
---------
Overtuig jezelf ervan dat de data nog steeds goed is/klopt na het schudden.

---

Tot slot slaan we de data op voor hergebruik:

In [0]:
pickle_file = os.path.join(data_root, 'notMNIST.pickle')

try:
  f = open(pickle_file, 'wb')
  save = {
    'train_dataset': train_dataset,
    'train_labels': train_labels,
    'valid_dataset': valid_dataset,
    'valid_labels': valid_labels,
    'test_dataset': test_dataset,
    'test_labels': test_labels,
    }
  pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
  f.close()
except Exception as e:
  print('Kon het volgende niet opslaan', pickle_file, ':', e)
  raise

In [0]:
statinfo = os.stat(pickle_file)
print('Gecomprimeerde pickle grootte:', statinfo.st_size)

Compressed pickle size: 718193801


---
Probleem/vraag 5
---------
Door de samenstelling van deze dataset kunnen er veel overlappende metingen/samples zijn, inclusief trainingsgegevens die zowel in de validatie- als in de testset zitten! 

Overlap tussen training en test kan een vertekend beeld geven van de resultaten, vooral als je het model gebruik in een omgeving waar nooit overlap plaatsvindt, maar het eigenlijk goed als je verwacht dat de samples terugkomen bij gebruik.

Meet hoeveel de overlap tussen training, validatie en test-samples.

Optionele vragen:
- Hoe zit het met bijna duplicaten tussen datasets? (Beelden die bijna identiek zijn)
- Maak een schone/clean validatie- en testset en vergelijk de nauwkeurigheid met die in de daaropvolgende opdrachten.

---

---
Probleem/vraag 6
---------
Laten we een idee te krijgen van wat een `on-the-shelf` classificator kan opleveren met deze gegevens. Het is altijd goed om te checken of er iets is om te leren en dat het een probleem is dat niet zo triviaal is er al een kant-en-klare oplossing het oplosbaar maakt.

Train een simpel model op deze gegevens met behulp van 50, 100, 1000 en 5000 trainings samples. 

Hint: u kunt het LogisticRegression model van sklearn.linear_model gebruiken.

Optionele vraag: train een `on-the-shelf model` op alle gegevens!
